In [97]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 필요한 모듈 생성

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 데이터 불러오기

In [99]:
titanic = pd.read_csv('/content/drive/MyDrive/와이빅타 교육세션 유찬/titanic.csv')

In [100]:
print(titanic.shape)
titanic.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Feature Engineering

## 결측치 확인

In [50]:
# 두 코드 모두 가능!
titanic.info()
titanic.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 결측치 제거

In [51]:
titanic_1 = titanic.dropna()
print("결측치 제거 전 데이터 개수 : ", titanic.shape[0])
print("결측치 제거 후 데이터 개수 : ", titanic_1.shape[0])

결측치 제거 전 데이터 개수 :  891
결측치 제거 후 데이터 개수 :  183


In [52]:
# 결측치가 모두 사라진것을 볼 수 있음
titanic_1.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## 결측치 대체

In [53]:
####결측치를 0으로 대체####
titanic_2 = titanic.fillna(0)
print("결측치 대체 전 데이터 개수 : ", titanic.shape[0])
print("결측치 대체 후 데이터 개수 : ", titanic_2.shape[0])

결측치 대체 전 데이터 개수 :  891
결측치 대체 후 데이터 개수 :  891


- 결측치를 0으로 대체해줌으로써 데이터 개수는 기존 데이터와 같은것을 볼 수 있음

In [54]:
####결측치를 Mean or Median으로 대체####
titanic_3 = titanic.fillna(titanic.mean())
titanic_4 = titanic.fillna(titanic.median())
print(titanic_3.shape[0])
print(titanic_4.shape[0])

891
891


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
####결측치를 가장 빈도수가 높은 값으로 대체####
print(titanic['Age'].value_counts())
print(titanic['Cabin'].value_counts())
print(titanic['Embarked'].value_counts())

24.00    30
22.00    27
18.00    26
19.00    25
28.00    25
         ..
36.50     1
55.50     1
0.92      1
23.50     1
74.00     1
Name: Age, Length: 88, dtype: int64
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64
S    644
C    168
Q     77
Name: Embarked, dtype: int64


- 일단 결측치가 존재하는 Feature에서 어떤 unique값이 가장 빈도수가 높은지 찾아봄
    - Age : 24.00
    - Cabin : B96 B98
    - Embarked : S

In [107]:
# 빈도수가 높은 값으로 채워줌
titanic_5 = titanic.copy()
titanic_5.loc[titanic['Age'] != titanic['Age'],'Age'] = 24.00
titanic_5.loc[titanic['Cabin'] != titanic['Cabin'],'Cabin'] = 'B96'
titanic_5.loc[titanic['Embarked'] != titanic['Embarked'],'Embarked'] = 'S'
print("결측값 대체 전 Age의 24.00 값 개수 : ", titanic.Age.value_counts().iloc[0])
print("결측값 대체 후 Age의 24.00 값 개수 : ", titanic_5.Age.value_counts().iloc[0])

결측값 대체 전 Age의 24.00 값 개수 :  30
결측값 대체 후 Age의 24.00 값 개수 :  207


- 결측치를 빈도수가 가장 높았던 24.00으로 대체해줌으로써, 24.00값의 개수가 늘었음을 확인할 수 있음

In [57]:
####결측치를 임의의 값으로 대체####
titanic_6 = titanic.fillna(method = 'pad')
titanic_7 = titanic.fillna(method = 'bfill')
titanic_6.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C85,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C123,S


- pad : 특정한 수치를사용하지 않고, 결측치의 바로 앞에 있는 value를 채워넣는 방식
- bfill : pad와는 반대로 각 결측치의 바로 뒤의 값으로 채워넣는 방법

# 지표변수 (Indicator variables)

In [95]:
titanic['Age']

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [94]:
oldness = []
for a in titanic['Age']:
    if (a <= 30):
        oldness.append('young')
    elif (31 <= a <= 50):
        oldness.append('middle')
    else:
        oldness.append('old')

titanic['oldness'] = oldness

## Feature Split

In [108]:
first_name = []
last_name = []
titanic = titanic.astype({'Name': 'str'})
for d in titanic['Name'].str.split(","):
    first_name.append(d[1])
    last_name.append(d[0])
titanic['first_name'] = first_name
titanic['last_name'] = last_name

In [109]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,first_name,last_name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr. Owen Harris,Braund
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs. John Bradley (Florence Briggs Thayer),Cumings
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss. Laina,Heikkinen
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs. Jacques Heath (Lily May Peel),Futrelle
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr. William Henry,Allen


## 스케일링 Scaling

In [63]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

In [64]:
# MinMax Scaling
mm_scaler = mm_scaler.fit_transform(titanic['Age'].values.reshape(-1,1))
titanic['mm_scaled_age'] = mm_scaler

# Standard Scaling
sd_scaler = sd_scaler.fit_transform(titanic['Age'].values.reshape(-1,1))
titanic['sd_scaled_age'] = sd_scaler


In [65]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,oldness,mm_scaled_age,sd_scaled_age
0,1,0,3,2022-02-03,male,22.0,1,0,A/5 21171,7.2500,NaN,S,young,0.271174,-0.530377
1,2,1,1,2022-02-03,female,38.0,1,0,PC 17599,71.2833,C85,C,middle,0.472229,0.571831
2,3,1,3,2022-02-03,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,young,0.321438,-0.254825
3,4,1,1,2022-02-03,female,35.0,1,0,113803,53.1000,C123,S,middle,0.434531,0.365167
4,5,0,3,2022-02-03,male,35.0,0,0,373450,8.0500,NaN,S,middle,0.434531,0.365167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,2022-02-03,male,27.0,0,0,211536,13.0000,NaN,S,young,0.334004,-0.185937
887,888,1,1,2022-02-03,female,19.0,0,0,112053,30.0000,B42,S,young,0.233476,-0.737041
888,889,0,3,2022-02-03,female,NaN,1,2,W./C. 6607,23.4500,NaN,S,old,NaN,NaN
889,890,1,1,2022-02-03,male,26.0,0,0,111369,30.0000,C148,C,young,0.321438,-0.254825


## 라벨 인코딩 (Label Encoding)

In [111]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [112]:
encoded = encoder.fit_transform(titanic['Embarked'])

In [113]:
titanic['Embarked'] = encoded

In [114]:
titanic.Embarked.value_counts()

2    644
0    168
1     77
3      2
Name: Embarked, dtype: int64

In [115]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,first_name,last_name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,2,Mr. Owen Harris,Braund
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0,Mrs. John Bradley (Florence Briggs Thayer),Cumings
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,Miss. Laina,Heikkinen
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,2,Mrs. Jacques Heath (Lily May Peel),Futrelle
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,2,Mr. William Henry,Allen


## 원-핫 인코딩 (One-Hot Encoding)

In [72]:
pd.get_dummies(titanic['Embarked'])

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
886,0,0,1
887,0,0,1
888,0,0,1
889,1,0,0


#과제

##Titanic 데이터를 가지고 5 개의 FE 방법들 직접 사용해보기 (오늘 피피티에 소개되지 않은 FE 방법도 괜찮음)